In [ ]:
Stock = yf.Ticker("BCS")
Stock.info

In [ ]:
data= Stock.history(period="1y", interval ="1d")
print(data)

In [ ]:
data.head(25)

In [ ]:
data.isnull().sum()

In [ ]:
data.info()

In [ ]:
data["Daily_returns"]= data['Close'].pct_change(1)*100
filtered_data = data.loc["2025-05-01" : "2025-06-01"]
filtered_data.head(25)

In [ ]:
filtered_data['Daily_returns'].replace(np.nan,0, inplace=True)
filtered_data.head(25)
filtered_data.describe().round(2)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
filtered_data = filtered_data.reset_index()
filtered_data['Date'] = pd.to_datetime(filtered_data['Date'])
filtered_data

In [ ]:
fig=px.line(title="Barclays PLC Daily Returns")
fig.add_scatter(x=filtered_data['Date'], y=filtered_data['Daily_returns'], name='Daily Returns')


In [ ]:
def plot_financial_data(df, title):
    fig = px.line(title=title)
    
    for i in df.columns[1:]:
        fig.add_scatter(x=df['Date'], y=df[i], name=i)
        fig.update_traces(line_width=1.5)
        fig.update_layout(plot_bgcolor='white')
    fig.show()


In [ ]:
plot_financial_data(filtered_data[['Date','Close','Open','Low',"High"]], "Barclays PLC Daily Returns")


In [ ]:
plot_financial_data(filtered_data.iloc[:,[0,5]], "Barclays PLC Volume chart") 

In [ ]:
plot_financial_data(filtered_data.iloc[:,[0,8]], "Barclays PLC Daily Returns") 

In [ ]:
def percentage_return_classifier(percentage_return):
    
    if percentage_return > -0.3 and percentage_return <= 0.3:
        return 'Insignificant Change'
    elif percentage_return > 0.3 and percentage_return <= 3:
        return 'Positive Change'
    elif percentage_return > -3 and percentage_return <= -0.3:
        return 'Negative Change'
    elif percentage_return > 3 and percentage_return <= 7:
        return 'Large Positive Change'
    elif percentage_return > -7 and percentage_return <= -3:
        return 'Large Negative Change'
    elif percentage_return > 7:
        return 'Bull Run'
    elif percentage_return <= -7:
        return 'Bear Sell Off'

In [ ]:
filtered_data['Trend']= filtered_data['Daily_returns'].apply(percentage_return_classifier)
filtered_data

In [ ]:
trend_summary = filtered_data['Trend'].value_counts()
trend_summary

In [ ]:
plt.figure(figsize = (5, 5))
trend_summary.plot(kind = 'pie', y = 'Trend');

In [ ]:
filtered_data.set_index(['Date'], inplace=True)   
filtered_data

In [ ]:
# MULTIPLE STOCK DATA VISUALIZATION
import pandas as pd
import numpy as np

In [ ]:
close_price_df = pd.read_csv(r"C:\Users\ANIKET P DHONGDI\Downloads\stock_prices.csv")


In [ ]:
close_price_df


In [ ]:
daily_returns_df = close_price_df.iloc[:, 1:].pct_change(1) * 100
daily_returns_df.replace(np.nan, 0, inplace=True)
daily_returns_df 

In [ ]:

daily_returns_df.insert(0, 'Date', close_price_df['Date'])
daily_returns_df


In [ ]:
plot_financial_data(close_price_df, "Adjusted closing prices of multiple stocks")

In [ ]:
plot_financial_data(daily_returns_df, "Percentage daily returns of multiple stocks")

In [ ]:
fig= px.histogram(daily_returns_df.drop(columns=['Date']))
fig.update_layout(title_text='Distribution of Daily Returns for Multiple Stocks', xaxis_title='Daily Returns', yaxis_title='Frequency')

In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(daily_returns_df.drop(columns=['Date']).corr(), annot=True, cmap='coolwarm', fmt='.2f')

In [ ]:
sns.pairplot(daily_returns_df);

In [ ]:
def price_scaling(raw_price_df):
    scaled_price_df = raw_price_df.copy()
    for i in scaled_price_df.columns[1:]:
        scaled_price_df[i] = raw_price_df[i] / raw_price_df[i][0]
    return scaled_price_df

In [ ]:
price_scaling(close_price_df)

In [ ]:
import random 

def generate_portfoli_weights(n):
    weights= []
    for i in range(n):
        weights.append(random.random())
        
    weights=weights/np.sum(weights)
    return weights
        

In [ ]:
weight= generate_portfoli_weights(10)
print(weight)

In [ ]:
# Let's define the "weights" list similar to the slides
weights = [0.032266, 0.094461, 0.117917, 0.132624, 0.145942, 0.128299, 0.10009, 0.007403, 0.088581, 0.152417]
weights

In [ ]:
close_price_df


In [ ]:
portfolio_df = close_price_df.copy()
scaled_df = price_scaling(portfolio_df)
scaled_df

In [ ]:
initial_investment = 1000000
for i, stock in enumerate(scaled_df.columns[1:]):
    portfolio_df[stock] = weights[i] * scaled_df[stock]  * initial_investment
portfolio_df.round(1)

In [ ]:
def asset_allocation(df, weights, initial_investment):
    portfolio_df = df.copy()

    # Scale stock prices using the "price_scaling" function that we defined earlier (Make them all start at 1)
    scaled_df = price_scaling(df)
  
    for i, stock in enumerate(scaled_df.columns[1:]):
        portfolio_df[stock] = scaled_df[stock] * weights[i] * initial_investment

    # Sum up all values and place the result in a new column titled "portfolio value [$]" 
    # Note that we excluded the date column from this calculation
    portfolio_df['Portfolio Value [$]'] = portfolio_df[portfolio_df != 'Date'].sum(axis = 1, numeric_only = True)
            
    # Calculate the portfolio percentage daily return and replace NaNs with zeros
    portfolio_df['Portfolio Daily Return [%]'] = portfolio_df['Portfolio Value [$]'].pct_change(1) * 100 
    portfolio_df.replace(np.nan, 0, inplace = True)
    
    return portfolio_df

In [ ]:
n = len(close_price_df.columns)-1

# Let's generate random weights 
print('Number of stocks under consideration = {}'.format(n))
weights = generate_portfoli_weights(n).round(6)
print('Portfolio weights = {}'.format(weights))

# Let's test out the "asset_allocation" function
portfolio_df = asset_allocation(close_price_df, weights, 1000000)
portfolio_df.round(2)

In [ ]:
# Plot the portfolio percentage daily return
plot_financial_data(portfolio_df[['Date', 'Portfolio Daily Return [%]']], 'Portfolio Percentage Daily Return [%]')

# Plot each stock position in our portfolio over time
# This graph shows how our initial investment in each individual stock grows over time
plot_financial_data(portfolio_df.drop(['Portfolio Value [$]', 'Portfolio Daily Return [%]'], axis = 1), 'Portfolio positions [$]')

# Plot the total daily value of the portfolio (sum of all positions)
plot_financial_data(portfolio_df[['Date', 'Portfolio Value [$]']], 'Total Portfolio Value [$]')

In [ ]:
def simulation_engine(weights, initial_investment):

    portfolio_df = asset_allocation(close_price_df, weights, initial_investment)
  
    
    return_on_investment = ((portfolio_df['Portfolio Value [$]'][-1:] - 
                             portfolio_df['Portfolio Value [$]'][0])/ 
                             portfolio_df['Portfolio Value [$]'][0]) * 100
  
    #
    portfolio_daily_return_df = portfolio_df.drop(columns = ['Date', 'Portfolio Value [$]', 'Portfolio Daily Return [%]'])
    portfolio_daily_return_df = portfolio_daily_return_df.pct_change(1) 
  

    expected_portfolio_return = np.sum(weights * portfolio_daily_return_df.mean() ) * 252
  
    
    covariance = portfolio_daily_return_df.cov() * 252 
    expected_volatility = np.sqrt(np.dot(weights.T, np.dot(covariance, weights)))

    # Check out the chart for the 10-years U.S. treasury at https://ycharts.com/indicators/10_year_treasury_rate
    rf = 0.03 # Try to set the risk free rate of return to 1% (assumption)

    # Calculate Sharpe ratio
    sharpe_ratio = (expected_portfolio_return - rf)/expected_volatility 
    return expected_portfolio_return, expected_volatility, sharpe_ratio, portfolio_df['Portfolio Value [$]'][-1:].values[0], return_on_investment.values[0]


initial_investment = 1000000
portfolio_metrics = simulation_engine(weights,initial_investment)


print('Expected Portfolio Annual Return = {:.2f}%'.format(portfolio_metrics[0] * 100))
print('Portfolio Standard Deviation (Volatility) = {:.2f}%'.format(portfolio_metrics[1] * 100))
print('Sharpe Ratio = {:.2f}'.format(portfolio_metrics[2]))
print('Portfolio Final Value = ${:.2f}'.format(portfolio_metrics[3]))
print('Return on Investment = {:.2f}%'.format(portfolio_metrics[4]))


In [ ]:
#Set the number of simulation runs
sim_runs = 25000
initial_investment = 1000000

# Placeholder to store all weights
weights_runs = np.zeros((sim_runs, n))

# Placeholder to store all Sharpe ratios
sharpe_ratio_runs = np.zeros(sim_runs)

# Placeholder to store all expected returns
expected_portfolio_returns_runs = np.zeros(sim_runs)

# Placeholder to store all volatility values
volatility_runs = np.zeros(sim_runs)

# Placeholder to store all returns on investment
return_on_investment_runs = np.zeros(sim_runs)

# Placeholder to store all final portfolio values
final_value_runs = np.zeros(sim_runs)

for i in range(sim_runs):
    # Generate random weights 
    weights = generate_portfoli_weights(n)
    # Store the weights
    weights_runs[i,:] = weights
    
    # Call "simulation_engine" function and store Sharpe ratio, return and volatility
    # Note that asset allocation is performed using the "asset_allocation" function  
    expected_portfolio_returns_runs[i], volatility_runs[i], sharpe_ratio_runs[i], final_value_runs[i], return_on_investment_runs[i] = simulation_engine(weights, initial_investment)
    print("Simulation Run = {}".format(i))   
    print("Weights = {}, Final Value = ${:.2f}, Sharpe Ratio = {:.2f}".format(weights_runs[i].round(3), final_value_runs[i], sharpe_ratio_runs[i]))   
    print('\n')


In [ ]:
# Step 1: Find the index of the highest expected return
best_return_index = expected_portfolio_returns_runs.argmax()

# Step 2: Extract the weights and other stats
best_return = expected_portfolio_returns_runs[best_return_index]
best_weights = weights_runs[best_return_index]
best_final_value = final_value_runs[best_return_index]
best_sharpe = sharpe_ratio_runs[best_return_index]

# Step 3: Print the result
print("Highest Expected Return Portfolio")
print("Expected Return: {:.2f}%".format(best_return * 100))  # if return is in decimal
print("Final Portfolio Value: ${:,.2f}".format(best_final_value))
print("Sharpe Ratio: {:.2f}".format(best_sharpe))
print("Asset Weights: ", best_weights.round(3))


In [ ]:
sharpe_ratio_runs
sharpe_ratio_runs.argmax()
sharpe_ratio_runs.max()
weights_runs
weights_runs[sharpe_ratio_runs.argmax(), :]


In [ ]:
optimal_portfolio_return, optimal_volatility, optimal_sharpe_ratio, highest_final_value, optimal_return_on_investment = simulation_engine(weights_runs[sharpe_ratio_runs.argmax(), :], initial_investment)

In [ ]:
print('Best Portfolio Metrics Based on {} Monte Carlo Simulation Runs:'.format(sim_runs))
print('  - Portfolio Expected Annual Return = {:.02f}%'.format(optimal_portfolio_return * 100))
print('  - Portfolio Standard Deviation (Volatility) = {:.02f}%'.format(optimal_volatility * 100))
print('  - Sharpe Ratio = {:.02f}'.format(optimal_sharpe_ratio))
print('  - Final Value = ${:.02f}'.format(highest_final_value))
print('  - Return on Investment = {:.02f}%'.format(optimal_return_on_investment))

In [ ]:
# Create a DataFrame that contains volatility, return, and Sharpe ratio for all simualation runs
sim_out_df = pd.DataFrame({'Volatility': volatility_runs.tolist(), 'Portfolio_Return': expected_portfolio_returns_runs.tolist(), 'Sharpe_Ratio': sharpe_ratio_runs.tolist() })
sim_out_df

In [ ]:
# Plot volatility vs. return for all simulation runs
# Highlight the volatility and return that corresponds to the highest Sharpe ratio
import plotly.graph_objects as go
fig = px.scatter(sim_out_df, x = 'Volatility', y = 'Portfolio_Return', color = 'Sharpe_Ratio', size = 'Sharpe_Ratio', hover_data = ['Sharpe_Ratio'] )
fig.update_layout({'plot_bgcolor': "white"})
fig.show()


In [ ]:
# Let's highlight the point with the highest Sharpe ratio
fig = px.scatter(sim_out_df, x = 'Volatility', y = 'Portfolio_Return', color = 'Sharpe_Ratio', size = 'Sharpe_Ratio', hover_data = ['Sharpe_Ratio'] )
fig.add_trace(go.Scatter(x = [optimal_volatility], y = [optimal_portfolio_return], mode = 'markers', name = 'Optimal Point', marker = dict(size=[40], color = 'red')))
fig.update_layout(coloraxis_colorbar = dict(y = 0.7, dtick = 5))
fig.update_layout({'plot_bgcolor': "white"})
fig.show()